In [1]:
import numpy as np
import matplotlib.pyplot as plt
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
import sys
sys.path.append('/home/thayer/Desktop/DavidCS/ubuntu_partition/code/pydar/')
import pydar

%load_ext autoreload
%autoreload 2

pdal not imported, functions relying on it will fail
opencv was not imported, functions relying on it will fail


In [2]:
project_path = '/media/thayer/Data/mosaic_lidar/ROV/'
project_name = 'mosaic_rov_220220.RiSCAN.RiSCAN'

suffix='slfsnow'

project = pydar.Project(project_path, project_name, import_mode='read_scan',
                        class_list='all', las_fieldnames=['Points',
                                                            'Classification',
                                                            'PointId'],
                        suffix=suffix)

In [3]:
# Apply the sop transform and create pointwise z uncertainty
project.apply_transforms(['sop'])
project.create_z_sigma()

In [4]:
# Now create normalized heights
x0 = 40.05
y0 = 65.05
nx = 200
ny = 200
dx = 0.1
dy = 0.1

bounds = (x0-5, x0+nx*dx+5, y0-5, y0+ny*dy+5, -10, 0)
#project.create_empirical_cdf(bounds)
#project.create_normalized_heights()

In [8]:
# Display normalized heights

#project.display_project(-3, 3, field='norm_height')

In [5]:
# Examine the polydata we'll be using to create image
pdata = project.get_merged_points(x0=x0, y0=y0, wx=nx*dx, wy=ny*dy)
print(pdata)

vtkPolyData (0x55cf589a59b0)
  Debug: Off
  Modified Time: 156991680
  Reference Count: 1
  Registered Events: (none)
  Information: 0x55cf589a58b0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 156991684
  Field Data:
    Debug: Off
    Modified Time: 156991515
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 14380
  Number Of Cells: 7
  Cell Data:
    Debug: Off
    Modified Time: 156991646
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x55cf589a6010)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x55cf589a5cd0
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 1 1 1 1 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 1 1 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    

In [6]:
%%time

# Test our function for using gpytorch to create an image from points

yaw = 0

#leafsize = 500000
lengthscale = 2
outputscale = 0.01
nu = 1.5
n_neighbors = 256

project.merged_points_to_image(nx, ny, dx, dy, x0, y0, lengthscale, outputscale, nu, n_neighbors=n_neighbors)

CPU times: user 42.9 s, sys: 1.59 s, total: 44.4 s
Wall time: 15.4 s


In [7]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max)

In [8]:
z_min = 0
z_max = 0.25

project.display_image(z_min, z_max, field='z_ci')

In [13]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max, warp_scalars=True)

In [19]:
print(project.image)

vtkImageData (0x5600bbf5f7f0)
  Debug: Off
  Modified Time: 1099731304
  Reference Count: 1
  Registered Events: (none)
  Information: 0x5600543717b0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 1099686936
  Field Data:
    Debug: Off
    Modified Time: 1099683294
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 40000
  Number Of Cells: 39601
  Cell Data:
    Debug: Off
    Modified Time: 1099683297
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x5600c2374410)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x5600c29c51c0
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 1 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 1 1 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 1 1 1 )
    Scalars: (n

In [14]:
v_min = 0
v_max = 0.3

project.display_image(v_min, v_max, field='Elevation',warp_scalars=True, color_field='z_ci', show_points=True)

In [10]:
%%time
# Test max_pts

max_pts = 5000

project.merged_points_to_image(nx, ny, dx, dy, x0, y0, lengthscale, outputscale, nu, n_neighbors=n_neighbors,
                              max_pts=max_pts)

CPU times: user 20.7 s, sys: 886 ms, total: 21.6 s
Wall time: 10.4 s


In [11]:
# Repeat, now with corner_coords
corner_coords = np.array([[40, 65, 0],
                         [40, 85, 0],
                         [60, 85, 0]])
yaw = 0

#leafsize = 500000
lengthscale = 2
outputscale = 0.01
nu = 1.5
n_neighbors = 256

project.merged_points_to_image(nx, ny, dx, dy, x0, y0, lengthscale, outputscale, nu, n_neighbors=n_neighbors,
                              corner_coords=corner_coords)

pdal not imported, functions relying on it will fail
opencv was not imported, functions relying on it will fail


/home/thayer/anaconda3/envs/vtk_gpytorch/lib/python3.7/site-packages/gpytorch/models/exact_gp.py:275: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  GPInputWarning,


In [12]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max)

In [21]:
# Try saving and then reading from file
project.write_image(suffix=suffix, name='test')

In [22]:
# Delete the project
del project

In [23]:
# Create a new project and read the image
project_path = '/media/thayer/Data/mosaic_lidar/ROV/'
project_name = 'mosaic_rov_220220.RiSCAN.RiSCAN'

suffix='slfsnow'

project = pydar.Project(project_path, project_name, import_mode='empty',
                        suffix=suffix)

In [24]:
project.read_image(suffix=suffix, name='test')

In [25]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max, warp_scalars=True)